#Importing the required libraries

In [1]:
#using selenium library to automate the browser and test

from selenium import webdriver      #webdriver component provides a programing interface to work on     
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup       #to parse the html document and scrape the necessary data

import time

#Connecting to the drivers

In [2]:
serv_obj = Service("C:\Drivers\chromedriver-win64\chromedriver")        #using Service classes to manage the local drivers
serv_obj.start()

#WebScraping the data

In [22]:
driver = webdriver.Remote(serv_obj.service_url)     #Remote class is being used to control the browser by sending commands
driver.get("https://www.google.com/maps/@12.9052786,80.1093562,15z?entry=ttu")      #passing the url through get function
finding = driver.find_element(By.NAME,"q").send_keys("restaurants near me", Keys.ENTER)     #find_element function helps to locate the element in html
    #here we are locating the input cell by name from html and passing the input content through send keys function

time.sleep(5)       #providing the pausing time explicitly for the browser to load (could be adjusted as per requirements)

restaurants = []

def web_scrape(n):      #creating a function to webscrape - n represents the number of restaurants to be scraped
    for x in range(n):      #creating a loop to get details of restaurants one by one
        profiles = driver.find_elements(By.CLASS_NAME, "hfpxzc")        #locating the class of list of restaurants in results container
        profiles[x].click()         #indexing the restaurants list to click and get further details
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        name = soup.find('h1', class_='DUwDvf lfPIob').get_text().strip() if soup.find('h1', class_='DUwDvf lfPIob').get_text().strip() else 'N/A'
                #scraping the name of restaurant using class and tag name from html     #if condition to return 'n/a' if element is not available
        
        address = soup.find(lambda tag: tag.name=='button' and tag.has_attr('data-item-id') and tag['data-item-id']=='address').get_text().strip() if soup.find(lambda tag: tag.name=='button' and tag.has_attr('data-item-id') and tag['data-item-id']=='address').get_text().strip() else 'N/A'
        phone = soup.find(lambda tag: tag.name=='button' and tag.has_attr('data-tooltip') and tag['data-tooltip']=='Copy phone number').get_text().strip() if soup.find(lambda tag: tag.name=='button' and tag.has_attr('data-tooltip') and tag['data-tooltip']=='Copy phone number').get_text().strip() else 'N/A'
        map = soup.find(lambda tag: tag.name=='button' and tag.has_attr('data-item-id') and tag['data-item-id']=='oloc').get_text().strip() if soup.find(lambda tag: tag.name=='button' and tag.has_attr('data-item-id') and tag['data-item-id']=='oloc').get_text().strip() else 'N/A'
                #as address, contact details fall in a common class, we are extracting details using lambda function and unique attributes in html
        
        restaurants.append({'name': name, 'address' : address, 'phone' : phone, 'map': map})        #appending the details
        
        driver.back()       #to drive back to webpage of search results and to select the next restaurant
        time.sleep(5)
    
web_scrape(4)       #calling the function with required number of restaurants to be scraped

import sqlite3      #importing sqlite3 module to save the scraped data in a database
connection = sqlite3.connect("hema.db")     #connecting to database
cursor=connection.cursor()
sql_command=('''
        CREATE TABLE project (
            name TEXT,
            address TEXT,
            phone TEXT,
            map TEXT)''')
cursor.execute(sql_command)     #creating a table

for xyz in restaurants:         #inserting the data using for loop
    cursor.execute('''INSERT INTO project (name, address, phone, map) VALUES (?, ?, ?, ?)''' ,
                    (xyz['name'], xyz['address'], xyz['phone'], xyz['map']))

connection.commit()

cursor.execute('SELECT * FROM project')     
result = cursor.fetchall()      
for row in result:      #to display the results
    print(row)
connection.close()


driver.quit()       #closing the browser

('Sangeetha Veg Restaurant', '\ue0c8No.1, 596, Grand Southern Trunk Rd, New Perungalathur, Perungalathur, Tamil Nadu 600048', '\ue0b0044 4217 0304', '\uf186V3WQ+VV Perungalathur, Tamil Nadu')
('CHENNAI 45 FOOD PARK', '\ue0c8121, 4th street, West, CTO Colony, Lakshmipuram, Tambaram, Chennai, Tamil Nadu 600045', '\ue0b0081220 61115', '\uf186W3HX+3R Chennai, Tamil Nadu')
('ANYOS KITCHEN', '\ue0c820, MELENDAI STREET, Mudichur Rd, West Tambaram, Chennai, Tamil Nadu 600045', '\ue0b0091761 51461', '\uf186W4G3+5C Chennai, Tamil Nadu')
('Hotel Sri Saai Bhavan', '\ue0c8W483+FQ4 ताम्ब्रम - मुदिचुर, Oragadam Rd, ताम्ब्रम वेस्ट, Irumbuliyur, Chennai, Tamil Nadu 600045', '\ue0b0098408 33501', '\uf186W483+FQ Chennai, Tamil Nadu')
